# Lesson 1 Data Extraction Fundamentals

## Beatles Diskography

In [52]:
# Your task is to read the input DATAFILE line by line, and for the first 10 lines (not including the header)
# split each line on "," and then for each line, create a dictionary
# where the key is the header title of the field, and the value is the value of that field in the row.
# The function parse_file should return a list of dictionaries,
# each data line in the file being a single list entry.
# Field names and values should not contain extra whitespace, like spaces or newline characters.
# You can use the Python string method strip() to remove the extra whitespace.
# You have to parse only the first 10 data lines in this exercise,
# so the returned list should have 10 entries!
import os

DATADIR = "data"
DATAFILE = "beatles-diskography.csv"


def parse_file(datafile):
    data = []
    rows = []
    with open(datafile, "r") as f:
        for line in f:        
            line = line.split(',')
            for i in range(len(line)):
                line[i] = line[i].strip()
            rows.append(line)
        for j in range(10):
            data.append(dict(zip(rows[0],rows[j+1])))
        
        print data[0]
        print data[9]
    return data


def test():
    # a simple test of your implemetation
    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_file(datafile)
    #print d
    firstline = {'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
    tenthline = {'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10 July 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}

    assert d[0] == firstline
    assert d[9] == tenthline
    
test()

{'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
{'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10 July 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}


## ERCOT Hourly Load

In [66]:
#!/usr/bin/env python
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format

"""

import xlrd
import numpy as np
import pandas as pd
from zipfile import ZipFile
datafile = "./data/2013_ERCOT_Hourly_Load_Data.xls"


def open_zip(datafile):
    with ZipFile(datafile, 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    sheet_data = pd.DataFrame([[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)])

    maxcoast = sheet_data[1][1:].max()
    print maxcoast
    maxcoastidx = sheet_data[1][1:].idxmax()
    print maxcoastidx
    maxtime = xlrd.xldate_as_tuple(sheet.cell_value(maxcoastidx, 0), 0)
    print maxtime
    
    mincoast = sheet_data[1][1:].min()
    print mincoast
    mincoastidx = sheet_data[1][1:].idxmin()
    mintime = xlrd.xldate_as_tuple(sheet.cell_value(mincoastidx, 0), 0)
    print mintime
    
    meancoast = sheet_data[1][1:].mean()
    print meancoast

    ### other useful methods:
    #print "\nROWS, COLUMNS, and CELLS:"
    #print "Number of rows in the sheet:", 
    #print sheet.nrows
    #print "Type of data in cell (row 3, col 2):", 
    #print sheet.cell_type(3, 2)
    #print "Value in cell (row 3, col 2):", 
    #print sheet.cell_value(3, 2)
    #print "Get a slice of values in column 3, from rows 1-3:"
    #print sheet.col_values(3, start_rowx=1, end_rowx=4)

    # print "\nDATES:"
    # print "Type of data in cell (row 1, col 0):", 
    # print sheet.cell_type(1, 0)
    # exceltime = sheet.cell_value(1, 0)
    # print "Time in Excel format:",
    # print exceltime
    # print "Convert time to a Python datetime tuple, from the Excel float:",
    # print xlrd.xldate_as_tuple(exceltime, 0)
    
    
    data = {
            'maxtime': maxtime,
            'maxvalue': maxcoast,
            'mintime': mintime,
            'minvalue': mincoast,
            'avgcoast': meancoast
    }
    return data


def test():
    #open_zip(datafile)
    data = parse_file(datafile)

    assert data['maxtime'] == (2013, 8, 13, 17, 0, 0)
    assert round(data['maxvalue'], 10) == round(18779.02551, 10)


test()

18779.02551
5392
(2013, 8, 13, 17, 0, 0)
6602.113899
(2013, 2, 3, 4, 0, 0)
10976.9334607


## Musicbrainz

In [85]:
"""
To experiment with this code freely you will have to run this code locally.
Take a look at the main() function for an example of how to use the code. We
have provided example json output in the other code editor tabs for you to look
at, but you will not be able to run any queries through our UI.
"""
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    """
    This is the main function for making queries to the musicbrainz API. The
    query should return a json document.
    """
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    """
    This adds an artist name to the query parameters before making an API call
    to the function above.
    """
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    """
    After we get our output, we can use this function to format it to be more
    readable.
    """
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    """
    Below is an example investigation to help you get started in your
    exploration. Modify the function calls and indexing below to answer the
    questions on the next quiz.

    HINT: Note how the output we get from the site is a multi-level JSON
    document, so try making print statements to step through the structure one
    level at a time or copy the output to a separate output file. Experimenting
    and iteration will be key to understand the structure of the data!
    """

    # Query for information in the database about bands named Nirvana
    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    #pretty_print(results)

    # Isolate information from the 4th band returned (index 3)
    #print "\nARTIST:"
    #pretty_print(results["artists"][4])

    # Query for releases from that band using the artist_id
    artist_id = results["artists"][4]["id"]
    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]

    # Print information about releases from the selected band
    #print "\nONE RELEASE:"
    #pretty_print(releases[6], indent=2)

    release_titles = [r["title"] for r in releases]
    print "\nALL TITLES:"
    for t in release_titles:
        print t

if __name__ == '__main__':
    main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
requesting http://musicbrainz.org/ws/2/artist/5b11f4ce-a62d-471e-81fc-a69a8278c7da?fmt=json&inc=releases

ALL TITLES:
Never Mind the Bollocks Here’s Nirvana
Blew
Sliver
Bleach
Sliver
Love Buzz / Big Cheese
Smells Like Teen Spirit
Smells Like Teen Spirit
Here She Comes Now / Venus in Furs
Smells Like Teen Spirit
Smells Like Teen Spirit
Bleach
Sliver
Smells Like Teen Spirit
Blew
Candy / Molly’s Lips
Bleach
Sliver
Smells Like Teen Spirit
Nevermind
Bleach
Smells Like Teen Spirit
Bleach
Sliver
Nevermind


### Quiz

In [139]:
"""
To experiment with this code freely you will have to run this code locally.
Take a look at the main() function for an example of how to use the code. We
have provided example json output in the other code editor tabs for you to look
at, but you will not be able to run any queries through our UI.
"""
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    """
    This is the main function for making queries to the musicbrainz API. The
    query should return a json document.
    """
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    """
    This adds an artist name to the query parameters before making an API call
    to the function above.
    """
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    """
    After we get our output, we can use this function to format it to be more
    readable.
    """
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    """
    Below is an example investigation to help you get started in your
    exploration. Modify the function calls and indexing below to answer the
    questions on the next quiz.

    HINT: Note how the output we get from the site is a multi-level JSON
    document, so try making print statements to step through the structure one
    level at a time or copy the output to a separate output file. Experimenting
    and iteration will be key to understand the structure of the data!
    """

    # Query for information in the database about bands named First Aid Kit, Queen, Beatles, Nirvana One Direction
    results = query_by_name(ARTIST_URL, query_type["simple"], "One Direction")
    #pretty_print(results)

    # Determine number of bands named First Aid Kit
    #print "\nNumber of artists named First Aid Kit:"
    #nFAK = 0
    #for n in results["artists"]:
    #    if n['name'] == 'First Aid Kit':
    #        nFAK += 1
    #print nFAK
    
    # Determine begin-area name for Queen
    #print results["artists"][2]["begin-area"]["name"]
    
    # Determine Spanish alias for Beatles
    #for n in results["artists"][8]['aliases']:
    #    if n['locale'] == 'es':
    #        print n['name']
    
    # Determine disambiguation for Nirvana
    #print results['artists'][4]['disambiguation']
    
    # Determine, when One Direction was formed
    #print results['artists'][0]['life-span']['begin']
    pretty_print(results['artists'][0])
    
    
    


if __name__ == '__main__':
    main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AOne+Direction&fmt=json
{
    "aliases": [
        {
            "begin-date": null, 
            "end-date": null, 
            "locale": null, 
            "name": "1D", 
            "primary": null, 
            "sort-name": "1D", 
            "type": "Artist name"
        }
    ], 
    "area": {
        "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed", 
        "name": "United Kingdom", 
        "sort-name": "United Kingdom"
    }, 
    "begin-area": {
        "id": "f03d09b3-39dc-4083-afd6-159e3f0d462f", 
        "name": "London", 
        "sort-name": "London"
    }, 
    "country": "GB", 
    "id": "1a425bbd-cca4-4b2c-aeb7-71cb176c828a", 
    "life-span": {
        "begin": "2010-07", 
        "ended": null
    }, 
    "name": "One Direction", 
    "score": "100", 
    "sort-name": "One Direction", 
    "tags": [
        {
            "count": 2, 
            "name": "pop"
        }, 
        {
            "count": 1, 
   

# Solar and Wind Energy

In [56]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""
import csv
import os

DATADIR = "data"
DATAFILE = "745090.csv"


def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            data.append(row)
        #    print row
    name = data[0][1]
    data = data[2:]
    
    # Do not change the line below
    return (name, data)


def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"


if __name__ == "__main__":
    test()

# Time and Value

In [90]:
# -*- coding: utf-8 -*-
'''
Find the time and value of max load for each of the regions
COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character | as the delimiter.

An example output can be seen in the "example.csv" file.
'''

import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "./data/2013_ERCOT_Hourly_Load_Data.xls"
outfile = "./data/2013_Max_Loads.csv"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = []
    
    # YOUR CODE HERE
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)

    num_cols = sheet.ncols   # Number of columns
    for col_idx in range(1, num_cols-1):    # Iterate through columns
        col_max = 0
        for row_idx in range(1, sheet.nrows):  # Iterate through rows
            cell_obj = sheet.cell(row_idx, col_idx)  # Get cell object by row, col
            if cell_obj.value > col_max:
                col_max = cell_obj.value
                max_time = xlrd.xldate_as_tuple(sheet.cell(row_idx, 0).value, 0)
                #print max_time
        data.append((max_time,col_max))
    print data
    return data

def save_file(data, filename):
    # YOUR CODE HERE
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter='|')
        writer.writerow(data)
    
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()

[((2013, 8, 13, 17, 0, 0), 18779.025510000003), ((2013, 8, 5, 17, 0, 0), 2380.1654089999956), ((2013, 6, 26, 17, 0, 0), 2281.2722140000024), ((2013, 8, 7, 17, 0, 0), 1544.7707140000005), ((2013, 8, 7, 18, 0, 0), 24415.570226999993), ((2013, 8, 8, 16, 0, 0), 5494.157645), ((2013, 8, 8, 18, 0, 0), 11433.30491600001), ((2013, 8, 7, 17, 0, 0), 1862.6137649999998)]


AssertionError: 